In [2]:
import pandas as pd
import numpy as np

In [3]:
dataSet = pd.read_csv('dataSet/iris.data', header=None)
dataSet.columns = ['sepal_length', 'sepal_width','petal_length', 'petal_width', 'class']
dependsColumns = pd.get_dummies(dataSet['class'])
data = pd.concat((dataSet, dependsColumns), axis=1)
data = data.drop(['class'], axis=1).to_numpy()
np.random.shuffle(data)
X = data[:, :-3]
y = data[:, -3:]

In [4]:
def sigmoid(x):
    return 1.0 / (1.0+np.exp(-x))

In [5]:
def sigmoid_dx(x):
    return sigmoid(x) * (1.0-sigmoid(x))

In [6]:
def loss(x, y):
    return -np.mean(np.sum(y*np.log(x), axis=1))

In [7]:
def loss_dx(x, y):
    return (x - y) / x.shape[0]

In [13]:
def accuracy(x, y):
    x = np.argmax(x, axis=1)
    y = np.argmax(y, axis=1)
    return np.mean(x == y)

In [8]:
def softmax(x):
    exp_ = np.exp(x)
    return exp_ / np.sum(exp_, axis=0)

In [9]:
def feed_forward(X, W1, W2, b1, b2):
    z1 = np.matmul(W1, X.T) + b1
    a1 = sigmoid(z1)
    z2 = np.matmul(W2, a1) + b2
    a2 = softmax(z2)
    return a2.T

In [10]:
def train_batch(X, y, weight1, weight2, bias1, bias2, l_rate):
    z1 = np.matmul(weight1, X.T) + bias1
    a1 = sigmoid(z1)
    z2 = np.matmul(weight2, a1) + bias2
    a2 = softmax(z2)
    
    loss_prim = loss_dx(a2, y.T)
    
    grad_w2 = np.matmul(loss_prim, a1.T)
    grad_w1 = np.matmul(np.matmul(weight2.T, loss_prim) * sigmoid_dx(a1), X)
    weight2 -= grad_w2 * l_rate
    weight1 -= grad_w1 * l_rate

    grad_b2 = np.mean(loss_prim, axis=1, keepdims=True)
    grad_b1 = np.mean(np.matmul(weight2.T, loss_prim) * sigmoid_dx(a1), axis=1, keepdims=True)
    bias2 -= grad_b2 * l_rate
    bias1 -= grad_b1 * l_rate

def train_weights(X, y, weight1, weight2, bias1, bias2, l_rate, batch_size, epochs = 20):
    size = X.shape[0]
    for epoch in range(1, epochs+1):
        rest = 1 if size % batch_size != 0 else 0
        batches = size // batch_size + rest
        for batch in range(batches):
            batch_y = y[batch*batch_size : (batch+1)*batch_size, :]
            batch_X = X[batch*batch_size : (batch+1)*batch_size, :]
            train_batch(batch_X, batch_y, weight1, weight2, bias1, bias2, l_rate)
        x = feed_forward(X, weight1, weight2, bias1, bias2)
        loss_score = loss(x, y)
        acc_score = accuracy(x, y)
        print("Epoch:",epoch,"\tLoss:",loss_score,"\tAccurancy",acc_score)

In [17]:
weights1 = np.random.normal(scale=0.25, size=(4, 4))
weights2 = np.random.normal(scale=0.25, size=(3, 4))
def prepareWeightsAndBias():
    bias1 = np.zeros((4, 1))
    bias2 = np.zeros((3, 1))
    return weights1.copy(), weights2.copy(), bias1, bias2

## Optimization method: SGD

In [18]:
w1, w2, b1, b2 = prepareWeightsAndBias()
train_weights(X, y, w1, w2, b1, b2, l_rate=0.1, batch_size=1)

Epoch: 1 	Loss: 1.061750435776887 	Accurancy 0.3466666666666667
Epoch: 2 	Loss: 1.0646228551432755 	Accurancy 0.3466666666666667
Epoch: 3 	Loss: 1.0610151876990521 	Accurancy 0.34
Epoch: 4 	Loss: 0.9535478277163006 	Accurancy 0.6666666666666666
Epoch: 5 	Loss: 0.8207631971647938 	Accurancy 0.6666666666666666
Epoch: 6 	Loss: 0.7367822984316419 	Accurancy 0.6666666666666666
Epoch: 7 	Loss: 0.6833028510948636 	Accurancy 0.6666666666666666
Epoch: 8 	Loss: 0.6449791482335681 	Accurancy 0.68
Epoch: 9 	Loss: 0.6139555033726881 	Accurancy 0.7066666666666667
Epoch: 10 	Loss: 0.5879257926737885 	Accurancy 0.7333333333333333
Epoch: 11 	Loss: 0.5688195093414496 	Accurancy 0.7333333333333333
Epoch: 12 	Loss: 0.5542323628693572 	Accurancy 0.7533333333333333
Epoch: 13 	Loss: 0.5429107794354421 	Accurancy 0.7466666666666667
Epoch: 14 	Loss: 0.5345964743238276 	Accurancy 0.74
Epoch: 15 	Loss: 0.5284788362848958 	Accurancy 0.7533333333333333
Epoch: 16 	Loss: 0.5238352556083921 	Accurancy 0.74
Epoch: 17 

## Optimization method: Mini Batch GD

In [19]:
w1, w2, b1, b2 = prepareWeightsAndBias()
train_weights(X, y, w1, w2, b1, b2, l_rate=0.1, batch_size=20)

Epoch: 1 	Loss: 1.0613715179596395 	Accurancy 0.6666666666666666
Epoch: 2 	Loss: 1.049997433029863 	Accurancy 0.62
Epoch: 3 	Loss: 1.0561503785866369 	Accurancy 0.4266666666666667
Epoch: 4 	Loss: 0.9848235627182529 	Accurancy 0.6666666666666666
Epoch: 5 	Loss: 0.8593231168563943 	Accurancy 0.6666666666666666
Epoch: 6 	Loss: 0.7608741065490113 	Accurancy 0.6666666666666666
Epoch: 7 	Loss: 0.6899785104780487 	Accurancy 0.6666666666666666
Epoch: 8 	Loss: 0.6254373763982732 	Accurancy 0.72
Epoch: 9 	Loss: 0.5918416155129692 	Accurancy 0.72
Epoch: 10 	Loss: 0.5705574873301827 	Accurancy 0.72
Epoch: 11 	Loss: 0.5424763736200318 	Accurancy 0.7466666666666667
Epoch: 12 	Loss: 0.529112632109897 	Accurancy 0.7466666666666667
Epoch: 13 	Loss: 0.5075987103238472 	Accurancy 0.7666666666666667
Epoch: 14 	Loss: 0.503625250554478 	Accurancy 0.7666666666666667
Epoch: 15 	Loss: 0.4979492881936111 	Accurancy 0.7666666666666667
Epoch: 16 	Loss: 0.4978943771159976 	Accurancy 0.7533333333333333
Epoch: 17 	L

## Optimization method: GD

In [20]:
w1, w2, b1, b2 = prepareWeightsAndBias()
train_weights(X, y, w1, w2, b1, b2, l_rate=0.02, batch_size=150)

Epoch: 1 	Loss: 1.0900290708172338 	Accurancy 0.6
Epoch: 2 	Loss: 1.0853248595320948 	Accurancy 0.66
Epoch: 3 	Loss: 1.0786885371814192 	Accurancy 0.7533333333333333
Epoch: 4 	Loss: 1.0695414714771097 	Accurancy 0.82
Epoch: 5 	Loss: 1.058938432390778 	Accurancy 0.82
Epoch: 6 	Loss: 1.0495724755264912 	Accurancy 0.7733333333333333
Epoch: 7 	Loss: 1.043750180083453 	Accurancy 0.7066666666666667
Epoch: 8 	Loss: 1.0416586541469197 	Accurancy 0.62
Epoch: 9 	Loss: 1.0420886404034175 	Accurancy 0.5933333333333334
Epoch: 10 	Loss: 1.0438427796585126 	Accurancy 0.64
Epoch: 11 	Loss: 1.0462793757938686 	Accurancy 0.6666666666666666
Epoch: 12 	Loss: 1.049113919732586 	Accurancy 0.6666666666666666
Epoch: 13 	Loss: 1.0520396504952478 	Accurancy 0.6533333333333333
Epoch: 14 	Loss: 1.0544945607992413 	Accurancy 0.6266666666666667
Epoch: 15 	Loss: 1.0556118763852638 	Accurancy 0.6066666666666667
Epoch: 16 	Loss: 1.0542773258406875 	Accurancy 0.5933333333333334
Epoch: 17 	Loss: 1.0492718681677395 	Accu